# Wielowarstwowa sieć jednokierunkowa MLP w Tensorflow, Część I

## Wstęp:

Celem ćwiczenia jest wprowadzenie do biblioteki Tensorflow i  przypomnienie podstawowej sieci MLP i wpływu hiperparametrów na uczenie i jakość otrzymywanych wyników.
Sieć powinna rozwiązywać problem klasyfikacji obrazów ze zbioru  CIFAR-10.  
Należy zdefiniować architekturę modelu, funkcję celu i dostarczyć dane do sieci. (Tensorflow automatycznie oblicza pochodne funkcji celu).


Zadania na laboratorium będą oparte o interfejs Subclassing API. W notebooku znajduje się treść zadania wraz ze zdefiniowanymi klasami bazowymi i narzędziami pomocniczymi.

Paczki, które mogą być dodatkowo zastosowane:  
- numpy  
- scikit-learn (Metryki)  
- matplotlib, seaborn (Wykresy)  
- tqdm (Pasek postępu)


## Lista zadań (Część I): 
1. Wykorzystując klasę bazową zdefiniowaną w BaseLayer zaimplementuj warstwę w pełni połączoną. (Dodawanie zmiennych do warstwy odbywa się za pomocą metody add_weight). Źródło:  https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer) (0.5 pkt).

2. Wykorzystując klasę bazową zdefiniowaną w BaseModel i zaimplementowaną warstwę w pełni połączoną w zadaniu 1 zdefiniuj architekturę sieci, funkcję uczenia sieci i funkcję ewaluacji (0.5 pkt).  
    **Przetwarzanie wstępne**
    - Spłaszczenie obrazu z wymiaru 32x32x3 na wymiar 3072
    
   **Architektura modelu do implementacji**
    - Warstwa w pełni połączona z 128 neuronami i funkcją aktywacji ReLU
    - Warstwa w pełni połączona z 10 neuronami i funkcją aktywacji Softmax
    
    Funkcje aktywacji znajdują się w module tensorflow.keras.activation
    
   **Hiperparametry uczenia**
    - Wielkość paczki: 100
    - Optymalizator: Adam
    - Współczynnik uczenia: 0.01
    - Liczba epok: 10
    - Funkcja kosztu: tf.keras.losses.SparseCategoricalCrossentropy
    
 
    
    

3. Przedstaw wykres accuracy, krzywą funkcji kosztu i podaj macierz pomyłek (confusion matrix) (1 pkt)
4. Zwizualizuj kilka przykładów na klasę, dla których model podejmowal złą decyzję i przeanalizuj dlaczego (1 pkt)

Jakość analizy i realizacji (prawidłowość wniosków, klarowność prezentacji, rozumienie modelu, jakość kodu)  (2 pkt)  

### Ograniczenia

1. Zadania 1 i 2 muszą być oddane razem z Zadaniem 3

2. Wykorzystanie gotowych modułów implementujących warstwy sieci np. tensorflow.keras.layers i innych jest zabronione!


## TensorFlow

### Wcztywanie zbioru danych i mechanizm paczkowania danych

Zbiór CIFAR10 jest jednym ze zbiorów danych, dla których zdefiniowano interfejs do jego wczytywania w module `tensorflow.keras.datasets`. 

Wczytanie zbiorów wygląda następująco
```python
(x_train, y_train), (x_test, y_test) = tensorflow.keras.datasets.cifar10.load_data()
```

Normalizacja wartości piskeli 0-255 do 1

```python
x_train = x_train / 255.0
x_test = x_test / 255.0
```

### Paczkowanie danych 
Do iterowania zbioru danych i podzielenia na paczki (ang. batch) można wykorzystać interfejs zdefiniowany w `tensorflow.data.Dataset`

Przykładowy kod:
```python
# Utworzenie obiektu
train_dataset = tensorflow.data.Dataset.from_tensor_slices((x_train, y_train))
# Shuffle i zdefiniowanie rozmiaru paczki
# buffer_size - rozmiar bufora do przeprowadzenia operacji shuffle (Ważne!); 
# batch_size - rozmiar paczki;
train_dataset = train_dataset.shuffle(buffer_size=50000).batch(batch_size=100)

# Iterowanie po paczkach:
for x, y in train_dataset:
    Some code...
```

**Rozmiar bufora (buffer_size) wpływa na losowość całej operacji shuffle!**   
Szczegóły można znaleźć tutaj: [LINK](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#shuffle)



## Klasy bazowe

In [ ]:
import abc

import tensorflow as tf

class AbstractLayer(tf.keras.layers.Layer):
    """Abstract Layer."""
    
    def __init__(self):
        """Inits the class."""
        super(AbstractLayer, self).__init__()
    
    @abc.abstractmethod
    def call(self, inputs):
        """Makes forward pass of the layer"""
        pass


class AbstractModel(tf.keras.Model):
    """Abstract model."""

    def __init__(self):
        """Inits the class."""
        super(AbstractModel, self).__init__()

    @abc.abstractmethod
    def call(self, inputs, training=False):
        """Makes forward pass of the network."""
        pass

    @abc.abstractmethod
    def fit(self, **kwargs):
        """Implements learning loop for the model.
        
        kwargs can contain optional parameters such as 
        num_epochs, batch_size, etc.
        """
        pass

    @abc.abstractmethod
    def predict(self, x):
        """Predicts outputs based on inputs (x)."""
        pass



# Rozwiązanie